In [157]:
import glob
import os
import warnings
import numpy as np
import Bio
import pandas as pd
import requests
from Bio import SeqUtils
from Bio.PDB import PDBParser, PDBIO
from Bio.SeqUtils import seq1
from tqdm import tqdm
from tqdm.notebook import tqdm
import json
import re

pd.set_option('display.max_columns', 1000)

###
Followed from here: 🧬 14656 Unique Mutations+Voxel Features+PDBs 🧬

https://www.kaggle.com/code/vslaykovsky/14656-unique-mutations-voxel-features-pdbs?scriptVersionId=109568406

In [42]:
import gzip
from tqdm.auto import tqdm
import pandas as pd

tqdm.pandas()

In [8]:
protherm_file = "/projects/robustmicrob/jlaw/inputs/protherm/20230124-protherm-browse-download.tsv"
df_protherm = pd.read_csv(protherm_file, sep='\t', index_col=0)
df_protherm

,PROTEIN,UniProt_ID,PubMed_ID,SOURCE,LENGTH,PDB_wild,PDB_mutant,MUTATION,PDB_Chain_Mutation,MUTATED_CHAIN,NO_MOLECULE,SEC_STR,ASA,T_(C),pH,BUFFER_NAME,BUFFER_CONC,ION_NAME_1,ION_CONC_1,ADDITIVES,PROTEIN_CONC,MEASURE,METHOD,∆G_H2O_(kcal/mol),∆∆G_H2O_(kcal/mol),∆G_(kcal/mol),∆∆G_(kcal/mol),Tm_(C),∆Tm_(C),∆H_(kcal/mol),∆HvH_(kcal/mol),m_(kcal/mol/M),Cm_(M),∆Cp_(kcal/mol),STATE,REVERSIBILITY,KEY_WORDS,REFERENCE,AUTHOR,REMARKS,RELATED_ENTRIES
NO,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
31581,Histidine phosphocarrier protein (HPr) kinase/...,O34483,33810099,Bacillus subtilis,310,-,-,wild-type,-,-,-,-,NaN,25,7.0,Sodium phosphate,50 mM,-,-,Urea (1.8 μM),-,Circular dichroism,Urea,-,-,-,-,-,-,-,-,1.1 (0.2),-,-,-,No,circular dichroism; conformational stability; ...,Int J Mol Sci. 22(6):3231 (2021) PMID: 33810099,"Neira JL, Cámara-Artigas A, Hernández-Cifre JG...",Biophysical probe used is <λ>280. The <λ>280 i...,-
31582,Histidine phosphocarrier protein (HPr) kinase/...,O34483,33810099,Bacillus subtilis,310,-,-,wild-type,-,-,-,-,NaN,25,7.0,Sodium phosphate,50 mM,-,-,Urea ( 6 μM),-,Circular dichroism,Urea,-,-,-,-,-,-,-,-,1.7 (0.2),-,-,-,No,circular dichroism; conformational stability; ...,Int J Mol Sci. 22(6):3231 (2021) PMID: 33810099,"Neira JL, Cámara-Artigas A, Hernández-Cifre JG...",Biophysical probe used is <λ>280. The <λ>280 i...,-
31583,Histidine phosphocarrier protein (HPr) kinase/...,O34483,33810099,Bacillus subtilis,310,-,-,wild-type,-,-,-,-,NaN,25,7.0,Sodium phosphate,50 mM,-,-,Urea (11 μM),-,Circular dichroism,Urea,-,-,-,-,-,-,-,-,1.8 (0.9),-,-,-,No,circular dichroism; conformational stability; ...,Int J Mol Sci. 22(6):3231 (2021) PMID: 33810099,"Neira JL, Cámara-Artigas A, Hernández-Cifre JG...",Biophysical probe used is <λ>280. The <λ>280 i...,-
31584,Histidine phosphocarrier protein (HPr) kinase/...,O34483,33810099,Bacillus subtilis,310,-,-,wild-type,-,-,-,-,NaN,25,7.0,Sodium phosphate,50 mM,-,-,Urea (18 μM),-,Circular dichroism,Urea,-,-,-,-,-,-,-,-,1.4 (0.3),-,-,-,No,circular dichroism; conformational stability; ...,Int J Mol Sci. 22(6):3231 (2021) PMID: 33810099,"Neira JL, Cámara-Artigas A, Hernández-Cifre JG...",Biophysical probe used is <λ>280. The <λ>280 i...,-
31585,Histidine phosphocarrier protein (HPr) kinase/...,O34483,33810099,Bacillus subtilis,310,-,-,wild-type,-,-,-,-,NaN,25,7.0,Sodium phosphate,50 mM,-,-,Urea (1.8 μM),-,Circular dichroism,Urea,-,-,-,-,-,-,-,-,1.0 (0.1),-,-,-,No,circular dichroism; conformational stability; ...,Int J Mol Sci. 22(6):3231 (2021) PMID: 33810099,"Neira JL, Cámara-Artigas A, Hernández-Cifre JG...",Biophysical probe used is <λ>295. The <λ>295 i...,-
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
31576,Azurin,P00282,18293933,Pseudomonas aeruginosa (strain ATCC 15692 / DS...,148,5AZU,-,C132S (Based on UniProt),5azu_A:C112S,A,1,Coil,0.0,-15,7.5,Tris,30 mM,-,-,-,10 microM,Fluorescence,GdnHCl,10.3,4.4,-,-,-,-,-,-,5.34,1.93,-,-,Yes,"thermodynamic stability, low-temperature, prot...","BIOCHEMISTRY 47, 3322-3331 (2008) PMID: 18293933","Strambini GB, Gonnelli M.","In the presence of TMAO, trimethylamine N-oxid...","14511, 14512, 14513, 14514, 14515, 14516, 1451..."
31577,Azurin,P00282,18293933,Pseudomonas aeruginosa (strain ATCC 15692 / DS...,148,5AZU,-,C132S (Based on UniProt),5azu_A:C112S,A,1,Coil,0.0,-15,7.5,Tris,30 mM,-,-,-,10 microM,Fluorescence,GdnHCl,7.43,1.53,-,-,-,-,-,-,4.11,1.81,-,-,Yes,"thermodynamic stability, low-temperature, prot...","BIOCHEMISTRY 47, 3322-3331 (2008) PMID: 18293933","Strambini GB, Gonnelli M.","In the presence of betaine, VL = 0.53% (VL=[so...","14511, 14512, 14513, 14514, 14515, 14516, 1451..."
31578,Azurin,P00282,18293933,Pseudomonas aeruginosa (strain ATCC 15692 / DS...,148,5AZU,-,C132S (Based on UniProt),5azu_A:C112S,A,1,Coil,0.0,-15,7.5,Tris,30 mM,-,-,-,10 microM,Fluorescence,GdnHCl,6.45,0.55,-,-,-,-,-,-,3.52,1.83,-,-,Yes,"thermodynamic st

In [109]:
df_protherm = df_protherm.dropna(subset='UniProt_ID')
print(len(df_protherm))

29410


In [28]:
# copied from here: https://stackoverflow.com/a/29805905/7483950
def read_fasta(fp, gzipped=False):
        name, seq = None, []
        for line in fp:
            if gzipped:
                line = line.decode()
            line = line.rstrip()
            if line.startswith(">"):
                if name: yield (name, ''.join(seq))
                name, seq = line[1:], []
            else:
                seq.append(line)
        if name: yield (name, ''.join(seq))

In [34]:
prot_set = set(df_protherm.UniProt_ID.dropna().unique())

In [35]:
len(prot_set)

928

In [43]:
fasta_file = "/projects/bpms/jlaw/inputs/uniprot/2023-01/uniprot_sprot_shortid.fasta.gz"
print(f"reading {fasta_file}")
seqs = {}
# prot_set = set(tms.keys())
with gzip.open(fasta_file, 'r') as f:
    for name, seq in tqdm(read_fasta(f, gzipped=True)):
        # just read in the proteins that have GO annotations
        if name not in prot_set:
            continue
        # make sure there aren't these strange AAs
        seq = seq.replace('B', 'D').replace('Z', 'E').replace('J', 'L')
        seqs[name] = seq

print(f"\t{len(seqs)} read")

reading /projects/bpms/jlaw/inputs/uniprot/2023-01/uniprot_sprot_shortid.fasta.gz



0it [00:00, ?it/s]
1it [00:00,  7.79it/s]
12814it [00:00, 67474.22it/s]
25861it [00:00, 94516.75it/s]
35913it [00:00, 86438.84it/s]
50066it [00:00, 104422.30it/s]
62213it [00:00, 109843.77it/s]
76068it [00:00, 118809.81it/s]
91314it [00:00, 129187.69it/s]
105378it [00:00, 132690.09it/s]
118809it [00:01, 104784.15it/s]
132688it [00:01, 113446.42it/s]
146270it [00:01, 119413.52it/s]
158899it [00:01, 121114.47it/s]
173531it [00:01, 128231.94it/s]
188565it [00:01, 134586.27it/s]
202675it [00:01, 136480.55it/s]
216544it [00:01, 136471.46it/s]
230496it [00:01, 137368.78it/s]
244343it [00:02, 132351.04it/s]
257691it [00:02, 130418.68it/s]
270813it [00:02, 128827.49it/s]
284792it [00:02, 131995.65it/s]
300117it [00:02, 138188.94it/s]
313988it [00:02, 137768.82it/s]
327801it [00:02, 126296.60it/s]
344037it [00:02, 136343.40it/s]
358304it [00:02, 138140.87it/s]
382244it [00:03, 167421.08it/s]
403890it [00:03, 181752.38it/s]
423490it [00:03, 185947.00it/s]
442228it [00:03, 176166.22it/s]
460047i

	812 read


In [48]:
print(len([s for s in prot_set - set(seqs.keys())]))
','.join([s for s in prot_set - set(seqs.keys())])
    

116


'Q7YUF0,O59170,A0A060GYS1,E5BBQ3,Q54276,Q47471,P01607,O00095,Q9REI6,P04156 ,Q7Z7M3,G7M0X5,O68541,O25949,I7H6W6,Q4QII4,Q7M089,Q7SIB6,O49003,A0A0H2Z8G0,Q5DVG3,Q8NJZ1,Q6T418,Q9WZ62,A2RJ53,Q7LZZ4,Q6B184,D0WVP7,K0A689,O66015,Q96HL2,Q9S446 ,Q5NV88,Q5SLE7,Q9NAV7,R4IZZ7,O06878,W0TJ64,Q8NJY3,P17670,Q7SID9,P02640 ,B8YLY0,Q9X4G3,Q75QQ5,P62990,O05387,Q59962,Q9R2S1,Q9NAV8,P02794 ,A0A1D5PBP6,O28558,K7N5M4,B3EYD3,Q8VP89,R9S082,Q9TQQ4,Q8GB52,A0A5K6WEI0,Q8GRA2,Q7SIG1,Q9S8W1,Q9EYL5,Q8RR56,S4WCF9,Q72SM7,Q7SSI0,A0A410ZNC6,Q56013,O66926,Q2XSL7,Q8AXI4,O26594,Q93D82,Q2XSK9,Q91472,Q7M1T4,C1DTU5,Q3HWZ5,Q9R782,-,P96988,Q40059,Q5KZ26,Q1J0W0,Q8VD40,B8C6L5,P60484-2,A5H7H0,Q56077,Q93P54,Q58NA1,Q8NJY6,A0A7H0P088,Q8GJ67 ,Q8VTF1,O61594,Q8ENY1,Q6VS32,P04745,Q3LUG9,P62988,Q53W59,P95467,Q2YD82,Q8NEZ4 ,Q25252,Q9WMV8,A0A1U8PF82,E5FNM0,A0A1E7EP28,Q56QW9,H9CWR6,P33120,Q95HA2'

In [50]:
# downloaded the rest of the IDs from uniprot
seqs_file2 = "/projects/robustmicrob/jlaw/inputs/protherm/20230124_extra_uniprot_seqs.tsv"
df_seqs = pd.read_csv(seqs_file2, sep='\t')
df_seqs.head(2)

,From,Entry,Reviewed,Entry Name,Protein names,Gene Names,Organism,Length,Sequence
0,Q7YUF0,Q7YUF0,unreviewed,Q7YUF0_TRYCR,Adenosylhomocysteinase (EC 3.3.1.1),SAHH C3747_17g594 ECC02_006339,Trypanosoma cruzi,437,MTDYKVRDISLAEWGRKAIEIAENEMPGLMELRREYSQSKPLKGAK...
1,O59170,O59170,unreviewed,O59170_PYRHO,438aa long hypothetical aminotransferase,PH1501,Pyrococcus horikoshii (strain ATCC 700860 / DS...,438,MLILNNMKANDIIKRYSKVIAQANKMHYFPLVPVKAENAKVWDING...


In [72]:
df_seqs[df_seqs.From == "P33120"]

,From,Entry,Reviewed,Entry Name,Protein names,Gene Names,Organism,Length,Sequence
121,P33120,P0DJL7,reviewed,DTXR_CORDI,Diphtheria toxin repressor (Iron-dependent dip...,dtxR DIP1414,Corynebacterium diphtheriae (strain ATCC 70097...,226,MKDLVDTTEMYLRTIYELEEEGVTPLRARIAERLEQSGPTVSQTVA...
122,P33120,H2I233,reviewed,DTXR_CORDW,Diphtheria toxin repressor (Iron-dependent dip...,dtxR CDPW8_1404,Corynebacterium diphtheriae (strain PW8),226,MKDLVDTTEMYLRTIYELEEEGVTPLRARIAERLEQSGPTVSQTVA...


In [73]:
df_protherm[df_protherm.UniProt_ID == "P33120"]

,PROTEIN,UniProt_ID,PubMed_ID,SOURCE,LENGTH,PDB_wild,PDB_mutant,MUTATION,PDB_Chain_Mutation,MUTATED_CHAIN,NO_MOLECULE,SEC_STR,ASA,T_(C),pH,BUFFER_NAME,BUFFER_CONC,ION_NAME_1,ION_CONC_1,ADDITIVES,PROTEIN_CONC,MEASURE,METHOD,∆G_H2O_(kcal/mol),∆∆G_H2O_(kcal/mol),∆G_(kcal/mol),∆∆G_(kcal/mol),Tm_(C),∆Tm_(C),∆H_(kcal/mol),∆HvH_(kcal/mol),m_(kcal/mol/M),Cm_(M),∆Cp_(kcal/mol),STATE,REVERSIBILITY,KEY_WORDS,REFERENCE,AUTHOR,REMARKS,RELATED_ENTRIES
NO,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
8779,Diphtheria toxin,P33120,2271548,Corynebacterium diphtheriae (strain PW8),226,1DDN,-,wild-type,-,-,2,-,NaN,-,8.0,phosphate,50 mM,-,-,"NaN3 (0.01%),",-,DSC,Thermal,-,-,-,-,44.2,-,110,-,-,-,3.5,-,yes,solute perturbation analysis; Structural stabi...,"BIOCHEMISTRY 29, 8677-8683 (1990) PMID: 2271548","Ramsay G, Freire E",NaN3(0.01%) was added in the experiment,"18522, 18523"
8780,Diphtheria toxin,P33120,2271548,Corynebacterium diphtheriae (strain PW8),226,1DDN,-,wild-type,-,-,2,-,NaN,-,8.0,phosphate,50 mM,-,-,"NaN3 (0.01%),",-,DSC,Thermal,-,-,-,-,57.9,-,190,-,-,-,4.3,-,yes,solute perturbation analysis; Structural stabi...,"BIOCHEMISTRY 29, 8677-8683 (1990) PMID: 2271548","Ramsay G, Freire E",NaN3(0.01%) was added in the experiment,"18522, 18523"


In [64]:
df_seqs[df_seqs.Entry == "P0CG48"].Sequence.values[0][633]

'V'

In [71]:
df_seqs.From.value_counts().head(10)

P62990    4
P62988    4
P04745    3
Q6B184    2
P17670    2
P33120    2
Q40059    1
P96988    1
Q9R782    1
Q3HWZ5    1
Name: From, dtype: int64

In [76]:
update_mappings = dict(zip(df_seqs.From, df_seqs.Entry))
update_mappings.update({'P62990': 'P0CH28',
             'P62988': 'P0CG48',
             'P04745': 'P0DUB6',
             'Q6B184': 'P0CY08',
             'P17670': 'P9WGE6',
             'P33120': 'H2I233',
            })

In [79]:
seqs.update(dict(zip(df_seqs.Entry, df_seqs.Sequence)))

In [78]:
df_protherm.replace(to_replace=update_mappings, inplace=True)
df_protherm[df_protherm.UniProt_ID == "H2I233"]

,PROTEIN,UniProt_ID,PubMed_ID,SOURCE,LENGTH,PDB_wild,PDB_mutant,MUTATION,PDB_Chain_Mutation,MUTATED_CHAIN,NO_MOLECULE,SEC_STR,ASA,T_(C),pH,BUFFER_NAME,BUFFER_CONC,ION_NAME_1,ION_CONC_1,ADDITIVES,PROTEIN_CONC,MEASURE,METHOD,∆G_H2O_(kcal/mol),∆∆G_H2O_(kcal/mol),∆G_(kcal/mol),∆∆G_(kcal/mol),Tm_(C),∆Tm_(C),∆H_(kcal/mol),∆HvH_(kcal/mol),m_(kcal/mol/M),Cm_(M),∆Cp_(kcal/mol),STATE,REVERSIBILITY,KEY_WORDS,REFERENCE,AUTHOR,REMARKS,RELATED_ENTRIES
NO,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
8779,Diphtheria toxin,H2I233,2271548,Corynebacterium diphtheriae (strain PW8),226,1DDN,-,wild-type,-,-,2,-,NaN,-,8.0,phosphate,50 mM,-,-,"NaN3 (0.01%),",-,DSC,Thermal,-,-,-,-,44.2,-,110,-,-,-,3.5,-,yes,solute perturbation analysis; Structural stabi...,"BIOCHEMISTRY 29, 8677-8683 (1990) PMID: 2271548","Ramsay G, Freire E",NaN3(0.01%) was added in the experiment,"18522, 18523"
8780,Diphtheria toxin,H2I233,2271548,Corynebacterium diphtheriae (strain PW8),226,1DDN,-,wild-type,-,-,2,-,NaN,-,8.0,phosphate,50 mM,-,-,"NaN3 (0.01%),",-,DSC,Thermal,-,-,-,-,57.9,-,190,-,-,-,4.3,-,yes,solute perturbation analysis; Structural stabi...,"BIOCHEMISTRY 29, 8677-8683 (1990) PMID: 2271548","Ramsay G, Freire E",NaN3(0.01%) was added in the experiment,"18522, 18523"


In [ ]:
df_protherm

In [83]:
len(df_protherm.dropna(subset='MUTATION'))

31854

In [93]:
seqs['P34152'][928]

'T'

In [98]:
df_protherm.replace('-', np.nan, inplace=True)

In [102]:
print(len(df_protherm.dropna(subset=["∆G_(kcal/mol)", "∆∆G_(kcal/mol)", "Tm_(C)", "∆Tm_(C)"], how='all')))
print(len(df_protherm.dropna(subset=["∆G_(kcal/mol)", "∆∆G_(kcal/mol)", "Tm_(C)", "∆Tm_(C)"], how='all').UniProt_ID.unique()))

19686
736


In [111]:
df_protherm.MUTATION.value_counts()

wild-type                                       10466
Y925E(Based on UniProt and PDB)                   264
E3R(Based on UniProt and PDB)                      49
R96H(Based on UniProt and PDB)                     49
S48K(Based on UniProt and PDB)                     29
                                                ...  
A974V M988I V1007I V1021L (Based on UniProt)        1
C126A(Based on UniProt and PDB)                     1
E81Q (Based on UniProt)                             1
W74C (Based on UniProt)                             1
A45L(Based on UniProt and PDB)                      1
Name: MUTATION, Length: 8617, dtype: int64

In [105]:
df_protherm.MUTATION.value_counts().head(20)

wild-type                            12191
-(Based on UniProt and PDB)            663
Y925E(Based on UniProt and PDB)        264
E3R(Based on UniProt and PDB)           49
R96H(Based on UniProt and PDB)          49
S48K(Based on UniProt and PDB)          29
H206L (Based on UniProt)                28
H107Y(Based on UniProt and PDB)         25
C132S (Based on UniProt)                25
C33S C36S L79C (Based on UniProt)       23
V138G (Based on UniProt)                22
A82P(Based on UniProt and PDB)          22
I77A (Based on UniProt)                 21
I77V (Based on UniProt)                 21
T329A(Based on UniProt and PDB)         21
S44A(Based on UniProt and PDB)          21
M23L(Based on UniProt and PDB)          20
D114C (Based on UniProt)                20
E50K(Based on UniProt and PDB)          20
G203A(Based on UniProt and PDB)         20
Name: MUTATION, dtype: int64

In [107]:
df_protherm[(df_protherm.MUTATION != 'wild-type') & (df_protherm.MUTATION.apply(lambda x: 'Based on UniProt' not in x))].MUTATION

NO
31604         M225L (Based on Paper)
31605         M234L (Based on Paper)
31606         M243L (Based on Paper)
31608         M225L (Based on Paper)
31609         M234L (Based on Paper)
                    ...             
27228      1pga_A:M1Y (Based on PDB)
26993     1pga_A:E27F (Based on PDB)
29925     2imm_A:F32Y (Based on PDB)
29926    2imm_A:L106I (Based on PDB)
29932     2imm_A:P43S (Based on PDB)
Name: MUTATION, Length: 283, dtype: object

In [141]:
def get_sequence(row):
    if row.UniProt_ID not in seqs: 
        return np.nan, np.nan
    seq = seqs[row.UniProt_ID]
    if row.MUTATION == 'wild-type':
        return [], seq
    # TODO handle these cases better
    if 'Based on PDB' in row.MUTATION:
        return np.nan, np.nan
    muts = row.MUTATION.split('(')[0].replace(',',' ').replace(';',' ').split(' ')
    mutations = []
    mut_seq = list(seq)
    for mut in muts:
        if mut == '': continue
        try:
            pos = int(mut[1:-1])-1
        except ValueError:
            # print(row)
            # raise
            return "non-standard", np.nan
        wt_aa = mut[0]
        mut_aa = mut[-1]
        try:
            if wt_aa != seq[pos]:
                print(f"WARNING: {wt_aa =} != {seq[pos]} for {row.UniProt_ID} {pos}")
                return "mismatch", np.nan
        except IndexError:
            print(f"WARNING: {pos =}, {len(seq) =} for {row.UniProt_ID}")
            return "index_error", np.nan
        mutations.append((wt_aa, pos, mut_aa))
        mut_seq[pos] = mut_aa
    return mutations, ''.join(mut_seq)

test = df_protherm.tail(10).apply(get_sequence, axis=1, result_type='expand')
test

,0,1
NO,,
31571,"[(C, 131, S)]",MLRKLAAVSLLSLLSAPLLAAECSVDIQGNDQMQFNTNAITVDKSC...
31572,"[(C, 131, S)]",MLRKLAAVSLLSLLSAPLLAAECSVDIQGNDQMQFNTNAITVDKSC...
31573,"[(C, 131, S)]",MLRKLAAVSLLSLLSAPLLAAECSVDIQGNDQMQFNTNAITVDKSC...
31574,"[(C, 131, S)]",MLRKLAAVSLLSLLSAPLLAAECSVDIQGNDQMQFNTNAITVDKSC...
31575,"[(C, 131, S)]",MLRKLAAVSLLSLLSAPLLAAECSVDIQGNDQMQFNTNAITVDKSC...
31576,"[(C, 131, S)]",MLRKLAAVSLLSLLSAPLLAAECSVDIQGNDQMQFNTNAITVDKSC...
31577,"[(C, 131, S)]",MLRKLAAVSLLSLLSAPLLAAECSVDIQGNDQMQFNTNAITVDKSC...
31578,"[(C, 131, S)]",MLRKLAAVSLLSLLSAPLLAAECSVDIQGNDQMQFNTNAITVDKSC...
31579,[],MLRKLAAVSLLSLLSAPLLAAECSVDIQGNDQMQFNTNAITVDKSC...


In [135]:
seqs['R9S082'][130]

'A'

In [144]:
# now add the sequence column
results = df_protherm.apply(get_sequence, axis=1, result_type='expand')
df_protherm[['mutations', 'sequence']] = results

/tmp/ipykernel_17779/1362713955.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_protherm[['mutations', 'sequence']] = results
/tmp/ipykernel_17779/1362713955.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_protherm[['mutations', 'sequence']] = results


In [146]:
df_protherm.mutations.value_counts()

[]                            10437
mismatch                        776
[(Y, 924, E)]                   264
index_error                     141
[(E, 2, R)]                      49
                              ...  
[(A, 78, W)]                      1
[(A, 78, C)]                      1
[(A, 78, P)]                      1
[(F, 108, I), (A, 121, G)]        1
[(T, 66, Q)]                      1
Name: mutations, Length: 7738, dtype: int64

In [152]:
df_protherm[df_protherm.mutations == "non-standard"]

,PROTEIN,UniProt_ID,PubMed_ID,SOURCE,LENGTH,PDB_wild,PDB_mutant,MUTATION,PDB_Chain_Mutation,MUTATED_CHAIN,NO_MOLECULE,SEC_STR,ASA,T_(C),pH,BUFFER_NAME,BUFFER_CONC,ION_NAME_1,ION_CONC_1,ADDITIVES,PROTEIN_CONC,MEASURE,METHOD,∆G_H2O_(kcal/mol),∆∆G_H2O_(kcal/mol),∆G_(kcal/mol),∆∆G_(kcal/mol),Tm_(C),∆Tm_(C),∆H_(kcal/mol),∆HvH_(kcal/mol),m_(kcal/mol/M),Cm_(M),∆Cp_(kcal/mol),STATE,REVERSIBILITY,KEY_WORDS,REFERENCE,AUTHOR,REMARKS,RELATED_ENTRIES,mutations,sequence
NO,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2141,Lysozyme,P00720,3405287,Enterobacteria phage T4 (Bacteriophage T4),164,2LZM,172L,I3CBased on UniProt and PDB),2lzm_A:I3C,NaN,1,Helix,10.81,NaN,2.0,Unknown,NaN,KCl,0.2 M,NaN,NaN,CD,Thermal,NaN,NaN,NaN,NaN,46.7,4.8,NaN,NaN,NaN,NaN,NaN,NaN,yes,bacteriophage T4; hydrophobicity; protein fold...,"NATURE 334, 406-410 (1988) PMID: 3405287","Matsumura M., Becktel W.J. & Matthews B.W.",NaN,"3929, 3930, 3931, 3932, 3933, 3934, 3935, 3936...",non-standard,NaN
12761,Dienelactone hydrolase,P0A115,25600287,Pseudomonas knackmussii (strain DSM 6978 / LMG...,236,4u2b,NaN,Q35H F38L C123S Y145C E199G S208G and G211D(Ba...,"4u2b_A:Q35H,4u2b_A:F38L,4u2b_A:S123S,4u2b_A:Y1...",A,NaN,NaN,NaN,NaN,7.5,Hepes,50 mM,NaCl,75 mM,5% DMSO,5 micro M,Activity,Thermal,NaN,NaN,NaN,NaN,51.2 (0.3),NaN,NaN,NaN,NaN,NaN,NaN,NaN,Unknown,Dienelactone hydrolase ; 3.10 helix; Urea dena...,"Protein J. 34(1), 82–89 (2015) PMID: 25600287","J. L. Porter, C. A. Collyer, and D. L. Ollis",The pseudo wild-type protein contains the muta...,"26659, 26660, 26661, 26662, 26663, 26664, 2666...",non-standard,NaN
12767,Dienelactone hydrolase,P0A115,25600287,Pseudomonas knackmussii (strain DSM 6978 / LMG...,236,4u2b,NaN,Q35H F38L C123S Y145C E199G S208G and G211D(Ba...,"4u2b_A:Q35H,4u2b_A:F38L,4u2b_A:S123S,4u2b_A:Y1...",A,NaN,NaN,NaN,NaN,7.5,Hepes,20 mM,NaN,NaN,1 mM EDTA,250 microM,fluorescence,Thermal,NaN,NaN,NaN,NaN,51.5 (0.5),NaN,NaN,NaN,NaN,NaN,NaN,NaN,Unknown,Dienelactone hydrolase ; 3.10 helix; Urea dena...,"Protein J. 34(1), 82–89 (2015) PMID: 25600287","J. L. Porter, C. A. Collyer, and D. L. Ollis",The pseudo wild-type protein contains the muta...,"26659, 26660, 26661, 26662, 26663, 26664, 2666...",non-standard,NaN
16269,Cold shock protein,P41016,11800561,Bacillus caldolyticus,66,1C9O,NaN,L66AL (Based on UniProt),1c9o_A:L66L,A,1,Coil,69.91,70,7.0,sodium cacodylate-HCl,100 mM,NaN,NaN,NaN,4 micro M,CD,Thermal,NaN,NaN,1.1,0.02,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,yes,cold shock protein; protein stability; coulomb...,J MOL BIOL 313 343-357 (2001) PMID: 11800561,"Perl D, Schmid FX",Elongation of one amino acid.,"2968, 2969, 2970, 2971, 2972, 2973, 2974, 2975...",non-standard,NaN


In [148]:
df_protherm.sequence.nunique()

9129

In [155]:
df_protherm['sequence'] = df_protherm.sequence.replace('B', 'D').replace('Z', 'E').replace('J', 'L')

/tmp/ipykernel_17779/3153238430.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_protherm['sequence'] = df_protherm.sequence.replace('B', 'D').replace('Z', 'E').replace('J', 'L')


In [156]:
seqs_to_write = df_protherm[['UniProt_ID', 'sequence']].dropna().drop_duplicates()
seqs_to_write.to_csv('/projects/robustmicrob/jlaw/inputs/protherm/sequences2.csv')

## Exploration


Looks like this is a hydrogen exchange experiment
- https://en.wikipedia.org/wiki/Hydrogen%E2%80%93deuterium_exchange#Protein_structure

In [90]:
df_protherm[df_protherm.MUTATION == "Y925E(Based on UniProt and PDB)"].head(2)

,PROTEIN,UniProt_ID,PubMed_ID,SOURCE,LENGTH,PDB_wild,PDB_mutant,MUTATION,PDB_Chain_Mutation,MUTATED_CHAIN,NO_MOLECULE,SEC_STR,ASA,T_(C),pH,BUFFER_NAME,BUFFER_CONC,ION_NAME_1,ION_CONC_1,ADDITIVES,PROTEIN_CONC,MEASURE,METHOD,∆G_H2O_(kcal/mol),∆∆G_H2O_(kcal/mol),∆G_(kcal/mol),∆∆G_(kcal/mol),Tm_(C),∆Tm_(C),∆H_(kcal/mol),∆HvH_(kcal/mol),m_(kcal/mol/M),Cm_(M),∆Cp_(kcal/mol),STATE,REVERSIBILITY,KEY_WORDS,REFERENCE,AUTHOR,REMARKS,RELATED_ENTRIES
NO,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
23202,Focal adhesion target domain,P34152,16909417,Mus musculus (Mouse),1052,1K40,-,Y925E(Based on UniProt and PDB),1k40_A:Y925E,A,1,Helix,58.49,25,5.2,NaAc,50 mM,-,-,-,0.5 mL,NMR Hydrogen exchange,GdnHCl,2.99,-,-,-,-,-,-,-,-,0,-,-,Unknown,native-state hydrogen exchange; partially unfo...,"PROTEINS 65, 259-265 (2006) PMID: 16909417","Zhou Z, Feng H, Bai Y.",The experiment was carried out for the FAT dom...,"14613, 14614, 14615, 14616, 14617, 14618, 1461..."
23203,Focal adhesion target domain,P34152,16909417,Mus musculus (Mouse),1052,1K40,-,Y925E(Based on UniProt and PDB),1k40_A:Y925E,A,1,Helix,58.49,25,5.2,NaAc,50 mM,-,-,-,0.5 mL,NMR Hydrogen exchange,GdnHCl,4.19,-,-,-,-,-,-,-,-,0,-,-,Unknown,native-state hydrogen exchange; partially unfo...,"PROTEINS 65, 259-265 (2006) PMID: 16909417","Zhou Z, Feng H, Bai Y.",The experiment was carried out for the FAT dom...,"14613, 14614, 14615, 14616, 14617, 14618, 1461..."


In [103]:
df_protherm[df_protherm.MUTATION == "Y925E(Based on UniProt and PDB)"].REMARKS.head(2).values

array(['The experiment was carried out for the FAT domain ( 900 to 1000). Global anfolding free energy, the results were obtained with the amide exchange at T 929 aminoacid.',
       'The experiment was carried out for the FAT domain ( 900 to 1000). Global anfolding free energy, the results were obtained with the amide exchange at V 932 aminoacid.'],
      dtype=object)